In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pickle
import itertools
from IPython.display import display

from tqdm import tqdm

import lightgbm as lgb
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from tools.get_describe_record import get_describe_record

In [2]:
raw_df = pd.read_excel("../data/raw/kadai.xlsx")
raw_df.head(2)
processe = raw_df.copy()
processe["batch_id"] = processe.groupby("process_end_time").ngroup()
processe.head()
tagged_anormaly = pd.read_csv("../data/processed/processe_tagged_anormaly.csv")
tagged_anormaly.head()
anomaly = tagged_anormaly[tagged_anormaly["is_anomaly"] == 1]
nomaly = tagged_anormaly[tagged_anormaly["is_anomaly"] == 0]
## 異常スパイク部分の予測モデル構築
train_df = anomaly[anomaly["batch_id"] < 75]
test_df = anomaly[anomaly["batch_id"] >= 75]
train_df = train_df.drop(columns=["is_anomaly","batch_OV_std", "batch_id", "process_end_time", "final_mes_time"])
test_df = test_df.drop(columns=["is_anomaly","batch_OV_std", "batch_id", "process_end_time", "final_mes_time"])

#Attention:------------------Attention:------------------Attention:------------------Attention:------------------Attention:------------------  
SENSOR_COLS = [f"X{i}" for i in [27, 30, 41, 25, 36]]
LAGS = [1, 2, 3, 5, 10, 20]
WINS = [3, 5, 10, 20]   # rolling window
#Attention:------------------Attention:------------------Attention:------------------Attention:------------------Attention:------------------  

def feature_engineering1(df, SENSOR_COLS=SENSOR_COLS, LAGS=LAGS, WINS=WINS):
    base = df[SENSOR_COLS].copy()
    feats = {}  # ★ここに全部貯める

    for c in SENSOR_COLS:
        s = df[c]

        # lag
        for l in LAGS:
            feats[f"{c}_lag{l}"] = s.shift(l)

        # diff / pct
        feats[f"{c}_diff1"] = s.diff(1)
        feats[f"{c}_diff2"] = s.diff(2)
        feats[f"{c}_pct1"]  = s.pct_change(1).replace([np.inf, -np.inf], np.nan)

        # rolling
        for w in WINS:
            r = s.rolling(w, min_periods=1)
            rmean = r.mean()
            rstd  = r.std()
            feats[f"{c}_rmean{w}"] = rmean
            feats[f"{c}_rstd{w}"]  = rstd
            feats[f"{c}_rmax{w}"]  = r.max()
            feats[f"{c}_rmin{w}"]  = r.min()
            feats[f"{c}_z{w}"]     = (s - rmean) / (rstd + 1e-9)
            feats[f"{c}_dev{w}"]   = s - rmean
        
        feats[f"{c}_energy"] = (
            feats[f"{c}_rstd20"] * feats[f"{c}_rmax20"]
        )

        feats[f"{c}_jump"] = s - s.shift(5)

    feat_df = pd.concat([base, pd.DataFrame(feats, index=df.index)], axis=1)
    feat_df = feat_df.ffill().bfill().fillna(0)
    return feat_df

def struct_model_spike(
    train_df,
    test_df,
    SENSOR_COLS = [f"X{i}" for i in [27, 30, 41, 25, 36]],
    LAGS = [1, 2, 3, 5, 10, 20],
    WINS = [3, 5, 10, 20]
):
    train_feature_df = feature_engineering1(train_df, SENSOR_COLS=SENSOR_COLS, LAGS=LAGS, WINS=WINS)
    test_feature_df = feature_engineering1(test_df, SENSOR_COLS=SENSOR_COLS, LAGS=LAGS, WINS=WINS)
    #train_feature_df.head()

    # spike用特徴量
    X_sp = train_feature_df
    y_sp = np.log1p(train_df["OV"].values)

    # 高OVを重くする（まずは80%点を基準）
    q = np.percentile(train_df["OV"].values, 90)  # “頂点”の境界
    w = np.ones_like(train_df["OV"].values, dtype=float)
    w[train_df["OV"].values >= q] = 1 + ((train_df["OV"].values[train_df["OV"].values >= q] / (q+1e-9))**3)
    w = np.clip(w, 1.0, np.percentile(w, 95))

    spike_predict_model = RandomForestRegressor(
        n_estimators=600,
        max_depth=18,
        min_samples_leaf=1,
        random_state=42,
        n_jobs=-1
    )

    spike_predict_model.fit(X_sp, y_sp, sample_weight=w)

    with open("../data/model/spike_predict_rf.pkl", "wb") as f:
        pickle.dump(spike_predict_model, f)


In [3]:
## 平常時の予測
feat =["process_end_time", "final_mes_time", "OV"]+  [f"X{i}" for i in range(1, 84)]

train_df = nomaly.loc[nomaly["batch_id"] < 75, feat]
test_df  = nomaly.loc[nomaly["batch_id"] >= 75, feat]


normal_features = ["X14", "X30", "X33", "X83"] + ["elapsed_day", "OV_lag1", "OV_diff", "OV_roll_mean3", "OV_roll_std3", "OV"]


def df_set_datetime(df, col_name):
    for col in col_name:
        df[col] = pd.to_datetime(df[col])
    return df

# process_end_timeを用いて経過時間, ラグを取得する。
def get_elapsed_day(df, base_time=None):
    if base_time == None:
        base_time = df['process_end_time'].min()
    df['elapsed_day'] = (df['process_end_time'] - base_time).dt.days
    return df

def set_LagOV(df,target="OV", lag_record_num=1, window=3):
    df[f"{target}_lag{lag_record_num}"] = df[target].shift(lag_record_num)
    df[f"{target}_diff"] = df[target].diff(1).shift(1)  
    df[f"{target}_roll_mean{window}"] = df[target].rolling(window).mean().shift(1)
    df[f"{target}_roll_std{window}"] = df[target].rolling(window).std().shift(1)
    #df = df.dropna().reset_index(drop=True)
    return df

def feature_engineering2(df, normal_features):
    df = df.copy()
    df = df_set_datetime(df, ["process_end_time", "final_mes_time"])
    df = get_elapsed_day(df)
    df = set_LagOV(df)
    df = df.drop(columns=[col for col in df.columns if col not in normal_features])
    return df

def struct_model_flat(
    train_df,
    test_df,
    normal_features,
):
    train_df = feature_engineering2(train_df, normal_features)
    test_df = feature_engineering2(test_df, normal_features)

    pipeline = Pipeline([
        #("scaler", StandardScaler()),
        ("scaler", StandardScaler()),
        ("model", LGBMRegressor(
            objective="regression",
            metric="rmse",
            verbose=-1,
            random_state=42
        ))
    ])

    train_X = train_df[normal_features].drop(columns=["OV"])
    pipeline.fit(train_X, train_df["OV"])

    with open("../data/model/non_spike_predict.pkl", "wb") as f:
        pickle.dump(pipeline, f)
    #test_X = test_df[normal_features].drop(columns=["OV"])
    #pred = pipeline.predict(test_X)
    #rmse = np.sqrt(mean_squared_error(test_df["OV"], pred))
    #print(f"RMSE (nomaly only): {rmse:.3f}")


In [4]:
# スパイク発生ロット
SENSOR_COLS = [f"X{i}" for i in [27, 30, 41, 25, 36]]
LAGS = [1, 2, 3, 5, 10, 20]
WINS = [3, 5, 10, 20]   # rolling window

# 通常ロット
normal_features = ["X14", "X30", "X33", "X83"] + ["elapsed_day", "OV_lag1", "OV_diff", "OV_roll_mean3", "OV_roll_std3"]
rows = []


In [5]:
def pred_all(
    processe,
    SENSOR_COLS = [f"X{i}" for i in [27, 30, 41, 25, 36]],
    LAGS = [1, 2, 3, 5, 10, 20],
    WINS = [3, 5, 10, 20],
    normal_features = ["X14", "X30", "X33", "X83"] + ["elapsed_day", "OV_lag1", "OV_diff", "OV_roll_mean3", "OV_roll_std3"]
):
    # 全体予測
    with open("../data/model/spike_detection_IFmodel.pkl", "rb") as f:
        spike_model = pickle.load(f)

    with open("../data/model/spike_predict_rf.pkl", "rb") as f:
        spike_predict_model = pickle.load(f)

    with open("../data/model/non_spike_predict.pkl", "rb") as f:
        non_spike_predict_model = pickle.load(f)

    with open("../data/score/IF_train_score.pkl", "rb") as f:
        score_train = pickle.load(f)
    thr = np.quantile(score_train, 0.90)

    for batch in range(75, 100):

        batch_df = processe[processe["batch_id"] == batch]
        batch_OV = batch_df["OV"].values

        # --- スパイク判定 ---
        desc_df = get_describe_record(df=processe, batch=batch)
        if hasattr(desc_df, "ndim") and desc_df.ndim == 1:
            desc_df = desc_df.to_frame().T

        score_test = -spike_model.score_samples(desc_df)
        is_spike = int((score_test >= thr).item())

        # --- 予測 ---
        if is_spike == 1:
            feature_df = feature_engineering1(
                batch_df,
                SENSOR_COLS=SENSOR_COLS,
                LAGS=LAGS,
                WINS=WINS
            )
            pred = spike_predict_model.predict(feature_df)
            pred = np.expm1(pred)
        else:
            feature_df = feature_engineering2(
                batch_df,
                normal_features
            )
            pred = non_spike_predict_model.predict(feature_df)

        # --- val_df に追加（★ここが重要） ---
        tmp_df = pd.DataFrame({
            "trueOV": batch_OV,
            "predOV": pred,
            "batch_id": batch,
            "is_spike": is_spike
        })
        rows.append(tmp_df)

    val_df = pd.concat(rows, ignore_index=True)

    rmse_all = np.sqrt(mean_squared_error(val_df["trueOV"], val_df["predOV"]))
    rmse_sp  = np.sqrt(mean_squared_error(val_df.loc[val_df["is_spike"]==1,"trueOV"],
                                        val_df.loc[val_df["is_spike"]==1,"predOV"]))
    rmse_no  = np.sqrt(mean_squared_error(val_df.loc[val_df["is_spike"]==0,"trueOV"],
                                        val_df.loc[val_df["is_spike"]==0,"predOV"]))

    #print(f"RMSE all   : {rmse_all:.3f}")
    #print(f"RMSE spike : {rmse_sp:.3f}")
    #print(f"RMSE normal: {rmse_no:.3f}")
    
    """plt.figure(figsize=(8, 5))
    mask_spike = val_df["is_spike"] == 1
    plt.plot(val_df.index[mask_spike], val_df["trueOV"][mask_spike], label="True OV(spike)", alpha=0.3)
    mask_normal = val_df["is_spike"] == 0
    plt.plot(val_df.index[mask_normal], val_df["trueOV"][mask_normal], label="True OV(normal)", alpha=0.3)
    plt.plot(val_df.index, val_df["predOV"], label="Predicted OV")

    plt.legend()
    plt.show()
    """
    return rmse_all, rmse_sp, rmse_no

In [ ]:
raw_df = pd.read_excel("../data/raw/kadai.xlsx")
raw_df.head(2)
processe = raw_df.copy()
processe["batch_id"] = processe.groupby("process_end_time").ngroup()
processe.head()
tagged_anormaly = pd.read_csv("../data/processed/processe_tagged_anormaly.csv")

features = [f"X{i}" for i in range(1, 84)]
comb_4A = list(itertools.combinations(features, 4))
comb_4B = list(itertools.combinations(features, 4))
best_rmse = 100

for combA in tqdm(comb_4A):
    for combB in comb_4B:
        try:
            anomaly = tagged_anormaly[tagged_anormaly["is_anomaly"] == 1]
            nomaly = tagged_anormaly[tagged_anormaly["is_anomaly"] == 0]
            ## 異常スパイク部分の予測モデル構築
            train_df = anomaly[anomaly["batch_id"] < 75]
            test_df = anomaly[anomaly["batch_id"] >= 75]
            train_df = train_df.drop(columns=["is_anomaly","batch_OV_std", "batch_id", "process_end_time", "final_mes_time"])
            test_df = test_df.drop(columns=["is_anomaly","batch_OV_std", "batch_id", "process_end_time", "final_mes_time"])

            train_df = processe[processe["batch_id"] < 75]
            test_df = processe[processe["batch_id"] >= 75]
            struct_model_spike(
                train_df,
                test_df,
                SENSOR_COLS = list(combA),
                LAGS = [1, 2, 3, 5, 10, 20],
                WINS = [3, 5, 10, 20]
            )
            
            feat =["process_end_time", "final_mes_time", "OV"]+  list(combB)
            train_df = nomaly.loc[nomaly["batch_id"] < 75, feat]
            test_df  = nomaly.loc[nomaly["batch_id"] >= 75, feat]
            normal_features = list(combB) + ["elapsed_day", "OV_lag1", "OV_diff", "OV_roll_mean3", "OV_roll_std3", "OV"]

            struct_model_flat(
                train_df,
                test_df,
                normal_features = list(combB) + ["elapsed_day", "OV_lag1", "OV_diff", "OV_roll_mean3", "OV_roll_std3", "OV"]
            )
            rmse_all, rmse_sp, rmse_no = pred_all(
                processe,
                SENSOR_COLS = list(combA),
                LAGS = [1, 2, 3, 5, 10, 20],
                WINS = [3, 5, 10, 20],
                normal_features = list(combB) + ["elapsed_day", "OV_lag1", "OV_diff", "OV_roll_mean3", "OV_roll_std3"]
            )
            print(
                f"combA: {combA}",
                f"combB: {combB}",
                f"RMSE all   : {rmse_all:.3f}",
                f"RMSE spike : {rmse_sp:.3f}",
                f"RMSE normal: {rmse_no:.3f}",
                sep="\t"
            )
            if best_rmse > rmse_all:
                best_rmse = rmse_all
                best_comb = (comb_4A, comb_4B, rmse_all, rmse_sp, rmse_no)
                with open("best_comb_temp.pkl", "wb") as f:
                    pickle.dump(best_comb, f)

            if rmse_no > 25:
                comb_4B.remove(combB)
                break
        except:
            pass
        if rmse_sp > 40:
            comb_4A.remove(combA)
            break
    if rmse_all < 40:
        best_comb = (comb_4A, comb_4B, rmse_all, rmse_sp, rmse_no)
        with open("best_comb.pkl", "wb") as f:
            pickle.dump(best_comb, f)
        break

                

  0%|          | 0/1837620 [00:00<?, ?it/s]